# Fakeit in PyXspec

This is an example of taking a type-II PHA file and using it as a 'dummy file' to trick Fakeit into producing type-II output files.
Type-II will hold all 10,000 synthetic spectra we want to produce, instead of only one spectra (type-I).


In order to trick Fakeit into outputting a type-II file, a type-II file must be read into XSPEC as a spectra.  Then we will read into XSPEC the spectra we wish to run Fakeit on.  We need to load this spectra because we need the program to read the STAT_ERR column and the POISSERR = T or = F  in both the source and background files.

In [1]:
from __future__ import division
from xspec import *
import os

In [2]:
dataDir = "/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/integrated/"
modDir  = "/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/PYXSPEC/GBMwLAT/grbm+powerlaw/"
os.chdir(dataDir)

In [3]:
Xset.chatter = 0   # TURN OFF XSPEC CHATTER.
AllData.clear()   # CLEAR ALL DATA, IF ANY.

### Load Data, dummy file and spectra.

s1 is the dummy type-II file.

s2 is the type-I file that we want to make 10,000 synthetic spectra of.

s1 is loaded into AllData(1) and s2 into AllData(2).  

In [4]:
#   DUMMY FILE, FORMAT IS TYPE-II !! 
s1              = Spectrum('bn080916009_LAT-LLE_srcspectra.pha{1}')
s1              = AllData(1)
s1.response     = 'bn080916009_LAT-LLE_weightedrsp.rsp{1}'
s1.background   = 'bn080916009_LAT-LLE_bkgspectra.bak{1}'


#   FORMAT FOR THIS FILE IS TYPE-I.
#   WE WANT 10,000 SYNTHETIC SPECTRA FOR THIS ONE:
s2              = Spectrum('glg_tte_n3_bn080916009_v01.pha1')
s2              = AllData(2)
s2.response     = 'glg_cspec_n3_bn080916009_v00.rsp2'
s2.background   = 'glg_tte_n3_bn080916009_v01.bak'

### Setting Model and Parameters

In [5]:
m1 = Model('grbm+powerlaw')

m1.grbm.alpha.values        = [-0.7429, 0.00742, -10.0, -3.0, 2.0, 5.0]
m1.grbm.beta.values         = [-2.177, 0.0217, -10.0, -5.0, 2.0, 10.0]
m1.grbm.tem.values          = [305.505, 3.055, 10.0, 30.0, 6000.0, 7000.0]
m1.grbm.norm.values         = [0.0190,0.00019, 1e-15, 1e-10, 1.0, 1.0]
m1.powerlaw.PhoIndex.values = [2.077, 0.020, -3.0, -2.0, 9.0, 10.0]
m1.powerlaw.norm.values     = [0.0009, 9.925e-06, 1e-15, 1e-10, 1.0, 1.0]

#  FREEZE ALL MODEL PARAMETERS.
for i in range(1, AllModels(1).nParameters+1):
    Parameter.frozen.fset(AllModels(1)(i), 1)

### Fakeit Settings

In [6]:
fs1 = FakeitSettings()
fs2 = FakeitSettings()
fs3 = FakeitSettings()

Here, we set up 3 Fakeit Settings because Fakeit will produce 1 file for each loaded spectra and then the remainder of the synthetic spectra will go into a third file.

If we want 10,000 spectra to go into ONE single type-II file, then we really need to tell Fakit to make 10,002 spectra.
s1 with fs1 settings goes into one file.  (1 spectra)
s2 with fs2 settings goes into a second file.  (1 spectra)
Remaining with fs3 settings goes into a third file.  (10,000 spectra)

You can choose to not make fs3 settings and by default the fs2 settings would be applied to all 10,000 spectra.  This sounds fine because that's what we want anyhow.  However, the issue occurs when naming the file.

Since XSPEC will make 3 files reguardless (1 for s1, 1 for s2, and then 1 for the remaining spectra), the third file created will write over the second file because it technically has the same name as the second one.  This causes issues with the creation of the third file and in some cases will either decide to make the third file or not.  So what will show up is either the 3rd file that wrote over the 2nd file and holds 10,000 spectra.  Or, the 3rd file never writes and you are left with the type-I file (the 2nd file produced) with only 1 spectra in it instead of all 10,000 like you wished.



It's best to just use the same method we do here:



#### fs1 are the settings from the loaded dummy file.
#### fs2 are the setting from the loaded spectral file in s2.
#### fs3, we specify those settings here.  They are technically the same as the fs2, except we use a different name to ouput the files to.  Instead of copying fs2 into fs3, we just set it up ourselves to ensure no issues occur.


    *** If the 'fileName' is not specified for any of the 
    Fakeit Settings, the prefix of the loaded spectral file 
    will be used.  Instead of having a bunch of files named 
    too similar to our spectral files, but ending in .fak, 
    we set up junk1 and junk2 as the filenames.

In [7]:
fs1.fileName = 'junk1.fak'
fs2.fileName = 'junk2.fak'

fs3.background ='glg_tte_n3_bn080916009_v01.bak'
fs3.response = 'glg_cspec_n3_bn080916009_v00.rsp2'
fs3.fileName = 'grbm+pow_n3.fak'

In [8]:
# MAKE A PYTHON LIST THAT HOLDS 10,000 COPIES OF THE fs3 SETTINGS
# TO GO WITH THE 10,000 SPECTRA IT WILL MAKE.
FS = [fs3]*10000

# THEN, INSERT THE SETTINGS FOR THE LOADED SPECTRA AT THE 
# BEGINNING OF THE FAKEIT SETTINGS LIST 'FS'
FS.insert(0, fs2)
FS.insert(0, fs1)

#### FS contains all the fakeit settings for the 10,002 spectra.
    len(FS) = 10002
#### The first two settings in the list should appear different from the remaining 10,000.
    FS[0:5]
    [<xspec.data.FakeitSettings at 0x10ce1c950>,
     <xspec.data.FakeitSettings at 0x112be7450>,
     <xspec.data.FakeitSettings at 0x112be74d0>,
     <xspec.data.FakeitSettings at 0x112be74d0>,
     <xspec.data.FakeitSettings at 0x112be74d0>]
    

In [9]:
print(FS[0].fileName, FS[1].fileName, FS[2].fileName)

('junk1.fak', 'junk2.fak', 'grbm+pow_n3.fak')


## Send it off to make spectra!

In [10]:
AllData.fakeit(len(FS), FS)